### JSALT'23 Computer Lab Part II

- In this part, we will use simple strategies to analyze and visualize human-to-human task-oriented conversations.

- For this, we will use a standard dataset called MultiWoz v2.2, that covers several task-oriented domains such as
restaurant, taxi, hotel, train bookings, tourist attractions, and even combination of these domains.

- However, we will use only a subset of the conversations and try to find a strucutre within the conversations

### Hardware
- We can use either Google collab or Kaggle notebooks or your personal laptops (with a GPU)
- Some part of the assignment requires a GPU

### Install required packages in a Kaggle notebook
```
! pip3 install sentence-transformers==2.2.2
! pip3 install datasets==2.12
! pip3 install scikit-learn
! pip3 install graphviz
! pip3 install matplotlib 
```

In [ ]:
! pip3 install sentence-transformers==2.2.2
! pip3 install datasets==2.12
! pip3 install scikit-learn
! pip3 install graphviz
! pip3 install matplotlib 

In [ ]:
import os
from typing import List
from copy import deepcopy
import tqdm

In [ ]:
import torch
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

In [ ]:
import numpy as np

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

from scipy import sparse

import graphviz  

import matplotlib.pyplot as plt

In [ ]:
def load_multiwoz_subset(domain):
    """Pick a subset of dialogues with given domain only.
    Since each dialogue can contain multiple domains, the selected
    subset will still be multi-domain with the given domain being dominant"""

    multiwoz_dset = load_dataset("multi_woz_v22")

    subset_ixs = []  # list
    subset_utts = []  # nested list
    
    services_selected = []
    
    for i, conv in enumerate(multiwoz_dset["train"]):
        # keys: dialogue_id, services, turns
        if domain in conv["services"]:
            utts = conv["turns"]["utterance"]
            subset_utts.append(utts)
            subset_ixs.append(i)
            services_selected.append("_".join(sorted(conv['services'])))
    
    return subset_utts, subset_ixs, services_selected

In [ ]:
# In Multiwoz dataset, conversations with a single domain rarely exist.
# Most of the convsersations involve multiple domains, hence "multi"-woz

domain = 'restaurant'

domain_convs, domain_ixs, services_selected = load_multiwoz_subset(domain)
print('Conversations with domain:', domain, '=', len(domain_convs))
print("Services selected\n ", "\n  - ".join(list(set(services_selected))))

utts_per_conv = np.asarray([len(conv) for conv in domain_convs])
print('\nTotal number of utterances (sentences):', np.sum(utts_per_conv))
avg_utts_per_conv = np.mean(utts_per_conv)
print('Average number of utts per conversation: {:.1f}'.format(avg_utts_per_conv))

In [ ]:
def extract_sentence_embeddings(sentences, model_name, out_emb_file, max_s=512):
    """Extract sentence embeddings using one of the models from Sentence_transformers
    See https://huggingface.co/sentence-transformers for list of supported models.
    
    Args:
    -----
        sentences (list): List of sentences where every sentence in a plain string
        model_name (str): Should match the name from sentence_transformers (eg: sentence-transformers/LaBSE)
        out_emb_file (str): File where the embeddings are saved, so that we can 
            load them from the disk
        max_s (int): Maximum sequence length, usually 512, but can be higher/lower of different models.
    """    

    if os.path.exists(out_emb_file):
        embs = np.load(emb_file)
        print('Re-loading sentence embeddings from file:', out_emb_file,)
        print('  Embeddings shape:', embs.shape)
        
        if embs.shape[0] != len(sentences):
            print("Loaded embeddings {:d} != number of sentences {:d}.".format(embs.shape[0], len(sentences)),
                  "Probably the embeddings loaded from the disk correspond to different set of sentences.")
            
    else:
        
        print("Extracting embeddings..")
        
        model = SentenceTransformer(model_name)
        
        # this should not exceed the max seq length that was used
        # while training the model
        model.max_sequence_length = max_s  
        # Consider using a GPU accelarator for this step
        # and uncomment the following line
        model.cuda()
        
        embs = model.encode(sentences)
        
        np.save(emb_file, embs)
        print("Saved embeddings", embs.shape, "to", emb_file)
        
    return embs
    

### Strategies to represent the dialogues within in conversation

1. Treat each utterance independent of others (naive)
2. Consider the `n` past history (or context) to represent current utterance.
3. Consider the `n` past and `n` future utterances.

#### Strategy 1: naive

In [ ]:
# we will flatten the conversations into independent sentences, but keeping the order
# We can try better strategies to obtain sentence embeddings, 
# eg: concatenating 1 or 2 previous sentences to the current one, etc.

# strategy 1: treat every sentence independent of each other
sentences_indep = []
for sents in domain_convs:
    sentences_indep.extend(sents)
print("Flattened sentences (independent):", len(sentences_indep))

model_name = "sentence-transformers/LaBSE"
emb_file = f"cluster_multiwoz/{domain}_sent_embs.npy"

embs_indep = extract_sentence_embeddings(sentences_indep, model_name, emb_file)

#### Strategy 2: Combine past utterance with the current one 

In [ ]:
# Running this cell will override the embeddings from previous cell
# appending previouse sentence to the current one and extracting embeddings
# sent_0, sent_0+sent_1, 
# Strategy 2: Prepend the current utterance with past utterance

sentences_hist1 = []
for sents in domain_convs:
    history = ""
    for i, sent in enumerate(sents):
        sentences_hist1.append(history + sent)        
        history = sent + " "
print("Sentences prepended with past 1 history:", len(sentences_hist1))

model_name = "sentence-transformers/LaBSE"
emb_file = f"cluster_multiwoz/{domain}_accum_h1_sent_embs.npy"

embs_hist1 = extract_sentence_embeddings(sentences_hist1, model_name, emb_file)

### Selecting a smaller subset from the `domain=restaurant`
- Skip this step for the first time.
- Try to visualize the clusters with all the conversations, later, pick only a subset and visualize again.
- In the following example, we will consider even a smaller subset, i.e., by selecting services that 
based on `restaurant` and `restaurant + taxi` booking.
- In reality, we can assume that we will have such kind of metadata available for conversations.

In [ ]:
# Select only embeddings that belong to few services
#
def select_subset_serivces(subset_services, embeddings):

    subset_sents = []
    subset_embs = []
    subset_domain_convs = []

    j = 0
    for i, serv in enumerate(services_selected):
        # print(i, serv, len(domain_convs[i]), "::", j, "->", j+len(domain_convs[i]))
        if serv in subset_services:
            # print(i, serv, "::", j, "->", j+len(domain_convs[i]), '::', embs[j:j+len(domain_convs[i])].shape)

            subset_domain_convs.append(domain_convs[i])
            subset_embs.append(embeddings[j:j+len(domain_convs[i]),])

            subset_sents.extend(domain_convs[i])

        j += len(domain_convs[i])
        
    return subset_domain_convs, subset_sents, subset_embs



subset_services = ("restaurant", "restaurant_taxi")

# we are can decide which embeddings to send
# embeddings from strategy 1 or strategy 2 or custom strategy..
subset_domain_convs, subset_sents, subset_embs = select_subset_serivces(subset_services, embs_hist1)

print(f"# Subset of domain {domain} conversations:", len(subset_domain_convs))
print("# Subset sentences:", len(subset_sents))

subset_embs = np.concatenate(subset_embs)
print('Corresponding subset embeddings:', subset_embs.shape)


In [ ]:
def compute_transitions(domain_convs: List[list], cluster_ixs: np.ndarray) -> np.ndarray:    
    """Compute the transitions from one cluster to the other, based on
    the actual conversation/dialogue flow. Also compute the assigment of sentence embeddings to 
    the cluster indices - mainly start of dialogue and end of dialogue sentences"""
    
    print('Computing transition matrix ..')
    n_clusters = np.unique(cluster_ixs).size
    print(' n_clusters', n_clusters)
    
    soc_clusters = []  # start of conversation clusters
    eoc_clusters = []  # end of conversation clusters
    # transitions: row(from) - col(to)
    transitions = np.zeros(shape=(n_clusters, n_clusters))
    
    i = 0    
    k = 0
    while i < len(domain_convs): # iterate over conversations/dialogs, where each conv is a seq of utts
        j = 0
        prev_cix = -1
        while j < len(domain_convs[i]):  # iterate over utts in the current dialogue
        
            cix = cluster_ixs[k]
            
            if j == 0:  # start of conversation
                soc_clusters.append(cix)
            else:
                # from, to
                transitions[prev_cix, cix] += 1                               
            
            prev_cix = deepcopy(cix)
            
            k += 1
            j += 1
            
        # end of conversation
        eoc_clusters.append(prev_cix)
        
        i += 1
        
    return transitions, np.asarray(soc_clusters), np.asarray(eoc_clusters)


def get_cluster_assignments(occurrences):
    """Get percentage of embeddings assigned to each cluster"""
    
    ixs, count = np.unique(occurrences, return_counts=True)
    max_ix = np.argmax(count)
    print(
        "Cluster ix:",
        np.array2string(ixs, precision=0, formatter={'int_kind': lambda x: "%4d" % x}, separator=' |')
    )
    print(
        "Percentage:",
        np.array2string(count * 100/count.sum(), precision=1, 
                        formatter={'float_kind': lambda x: "%4.1f" % x},
                        separator=' |')
    )
    print("{:.1f} % from cluster index {:d} is the most dense.".format(count[max_ix]*100./count.sum(), ixs[max_ix]))
    return count/count.sum(), ixs

#### Clustering with k-means
- Here we will cluster the subst of the embeddings using simple k-means clustering.
- You may also try other clustering strategies based on Gaussian Mixture Models or even neural network based models.

In [ ]:
# you may play with this number, but it is better to keep it between [avg_utts_per_conv/2, avg_utts_per_conv x 2]
print("Avg. utts per conversation: {:.0f}".format(avg_utts_per_conv))


def run_kmeans(n_clusters, embs, seed=12345):
    """Cluster the given embeddings into the specified number of clusters
    and return the hard cluster assignments"""

    print("Clustering", embs.shape[0], "embeddings into", n_clusters, "clusters with k-means..")

    norm_embs = preprocessing.normalize(embs)
    print("  Normalizing embeddings to have unit length..")

    kmeans = KMeans(n_clusters=n_clusters, random_state=seed, max_iter=1000)
    pred_ixs = kmeans.fit_predict(norm_embs)
    print('.. done')
    
    return pred_ixs


In [ ]:
pred_ixs = run_kmeans(12, embs_indep)  # kmeans with embeddings from strategy 1
# pred_ixs = run_kmeans(12, embs_hist1)  # kmeans with embeddings from strategy 2
# pred_ixs = run_kmeans(12, subset_embs)  # kmeans with subset of embeddings strategy 2

In [ ]:
# Check if the embeddings are more or less equally distributed

_ = get_cluster_assignments(pred_ixs)

In [ ]:
trans, soc, eoc = compute_transitions(domain_convs, pred_ixs)  # strategy 1
# trans, soc, eoc = compute_transitions(domain_convs, pred_ixs)  # strategy 2
# trans, soc, eoc = compute_transitions(subset_domain_convs, pred_ixs)  # strategy 2 with subset
print("Transition matrix:", trans.shape)
for i in range(trans.shape[0]):
    trans[i, :] /= trans[i, :].sum()
    
print("\nStart of conversation..")
# get the clusters where the conversation usually BEGIN
soc_prob, soc_ixs = get_cluster_assignments(soc)


print("\nEnd of conversation..")
# get the clusters where the conversation usually END
eoc_prob, eoc_ixs = get_cluster_assignments(eoc)

### Let us visualize the clusters and the transitions

In [ ]:
# Let us visualize the clusters and the transitions
# We can set thresholds on the transition scores which will enable to control
# the arcs that will be drawn

def visualize_graph(trans, soc_prob, eoc_prob):

    # see https://graphviz.readthedocs.io/en/stable/  for more documentation on graphviz

    dot = graphviz.Digraph(f"MutliWoz_{domain}", format='png', graph_attr={'rankdir':'LR'})  # initialize a graph
    
    n_clusters = trans.shape[0]

    # add BEGIN and END nodes
    dot.node('BEGIN', shape='doublecircle')
    dot.node('END', shape='doublecircle')

    # Add a node representing each cluster
    for i in range(n_clusters):
        dot.node(str(i), shape='circle')    

    # low threshold will show many arcs, higher threshold will only show dominant arcs
    s_thresh = 0.1  # threshold for drawing begin, end transitions
    thresh = 0.12  # threshold for drawing an intermediate arc

    # draw arrows from BEGIN to that cluster(s) where the start sentences live
    # given that they are above the `s_thresh`
    for i, prob in enumerate(soc_prob):
        if prob > s_thresh:
            dot.edge('BEGIN', str(soc_ixs[i]), label="{:.2f}".format(prob))
            dot.node(str(soc_ixs[i]), fillcolor='cyan', style='filled')

    # draw arrows from cluster(s) where the end sentences live to the END node
    for i, prob in enumerate(eoc_prob):
        if prob > s_thresh:
            dot.edge(str(eoc_ixs[i]), 'END', label="{:.2f}".format(prob))
            dot.node(str(eoc_ixs[i]), fillcolor='pink', style='filled')

    # draw intermediate arcs among the clusters where transitions > thresh

    for row_ix in range(trans.shape[0]):  # row_ix represent FROM, col_ixs represent TO
        row_trans = trans[row_ix, :] / trans[row_ix, :].sum()    
        thresh_ixs = np.where(row_trans > thresh)
        if thresh_ixs[0].size > 0:
            # print(row_ix, '->', thresh_ixs)
            for col_ix in thresh_ixs[0]:
                dot.edge(str(row_ix), str(col_ix), label="{:.2f}".format(row_trans[col_ix]))

    # show the image within the notebook
    # dot.view() will open the image in an external window
    return dot

In [ ]:
dot = visualize_graph(trans, soc_prob, eoc_prob)
dot

### What does each cluster (node) actually represent :
* Can we get any representative utterances from the dialogues ?
* Can we get any representative words (word cloud) for each cluster ?
* Can we automatically identify the underlying intents / slot values for each cluster ?

In [ ]:
def get_most_representative_words(sentences: list, pred_ixs: np.ndarray):
    """Get most representative words per each cluster, based on 
    relative counts normalized to yield probabilities
    """
    n_clusters = np.max(pred_ixs) + 1

    count_vect = CountVectorizer(stop_words='english', lowercase=True, strip_accents=None, min_df=2)
    counts = count_vect.fit_transform(sentences)

    print('counts (n_sentences x vocab_size):', counts.shape)
    int2vocab = {}
    for vocab, idx in count_vect.vocabulary_.items():
        int2vocab[idx] = vocab

    # cluster assignments to sentences in sparse format
    # shape = (n_clusters x n_sentences)
    sparse_cixs = sparse.csr_matrix((np.ones(len(pred_ixs,)), (pred_ixs, np.arange(len(pred_ixs)))))

    # cumulative word count per cluster
    wc_per_cluster = sparse_cixs.dot(counts)
    print('n_clusters x vocab_size:', wc_per_cluster.shape, type(wc_per_cluster))

    prob_per_cluster = wc_per_cluster / wc_per_cluster.sum(axis=0)

    # print(prob_per_cluster.shape)
    print()
    
    top_k = 15  # print TOP K words per cluster
    
    for i in range(n_clusters):
        sort_ixs = np.argsort(prob_per_cluster[i, :].A.squeeze())[::-1]
        print("= Cluster", i, end="=\n ")
        for j in range(top_k):
            print(int2vocab[sort_ixs[j]], end=", ")
        print("\n")


In [ ]:
get_most_representative_words(sentences_indep, pred_ixs)


In [ ]:
fig = plt.figure(num=1)

n_clusters = np.max(pred_ixs) + 1

# https://matplotlib.org/stable/tutorials/colors/colormaps.html
im = plt.imshow(trans, cmap='Purples')

plt.xticks(np.arange(n_clusters))
plt.yticks(np.arange(n_clusters))

plt.ylabel("\"From\" cluster")
plt.xlabel("\"To\" cluster")

plt.title("Transitions among clusters")

_ = plt.colorbar(im)

- At this point, you can stop and go back and try 
1. different strategies to represent sentences
2. and/or use different model to extract embeddings
3. and/or use different number of clusters or clustering techniques such as Agglomerative Hierarchical Clustering, etc.

# Advanced:
- Below, we can train a model to represent our data, i.e., sentences from one of the strategies.
- Then, we can extract most representative words per cluster.

### The following steps describe the generative process of our model 

1. Given a vocabulary of size $W$, and corresponding $k$ dimensional-word embeddings or a $k$-dimensional subspace defined by

    $\mathbf{E} \in \mathbb{R}^{W \times k}$

2. For each document $d=1 \ldots D$ 

    Sample a document-specific embedding as follows:

    $\mathbf{a}_d \sim \mathcal{N}(\mathbf{a}_d \mid \mathbf{0}, \mathbf{I})$
    
    Obtain the distribution of words in each document $\boldsymbol{\theta}_d$ by normalizing

    $\boldsymbol{\theta}_d = \mathrm{softmax}(\mathbf{b} + \mathbf{E} \, \mathbf{a}_d)$
    
    where $\mathbf{b}$ is a $V$ dimensional vector representing bias.    

    Generate a vector of words of a document (draw $N_d$ tokens independently) by sampling

    $\mathbf{x}_d \sim \mathrm{Multinomial}(\boldsymbol{\theta}_d, N_d) $
    
$\mathbf{x}_d$ for $d=1 \ldots D$ put row-wise in matrix $\mathbf{X}$ represents the document-by-word statistics of the dataset.

### Reality

In reality, we do not generate the data, instead, given the document-by-word counts $\mathbf{X}$ (rows represent document indices, and columns represent word indices), we would like to estimate the word embedding matrix $\mathbf{E}$, bias vector $\mathbf{b}$ and document embeddings $\mathbf{a}_d, \forall \, d=1 \ldots D$, that best explains (maxmizes the likelihood) the given (observed) counts $\mathbf{X}$.

### Likelihood

Since we assumed each document $d$ is sampled from a $\mathrm{Multonomial}$ distribution with parameters $\boldsymbol{\theta}_d$, we can compute the likelihood of a document as

$p(\mathbf{x}_d \mid \boldsymbol{\theta}_d) = \prod_{i=1}^{W} \theta_{di} ^ {x_{di}}$

or the log of the likelihood is

$
\log p(\mathbf{x}_d \mid \boldsymbol{\theta}_d) = \sum_{i} x_{di} \log (\theta_{di}) \\
\qquad \qquad \quad = \sum_{i} x_{di} \log \Big( \frac{\exp\{b_i + \mathbf{e}_i^{T} \mathbf{a}_d\}}{\sum_{j=1}^{W}\exp\{b_j + \mathbf{e}_j^{T} \mathbf{a}_d\}} \Big) \\
\qquad \qquad \quad = \sum_{i} x_{di} \Big[(b_i + \mathbf{e}_i^{T} \mathbf{a}_d) - \log \big(\sum_{j}\exp\{b_j + \mathbf{e}_j^{T} \mathbf{a}_d\} \big) \Big]
$


Log-likelihood for all the documents is the summation over individual log-likelihoods

$\mathcal{L} = \sum_{d=1}^{D} \log p(\mathbf{x}_d \mid \boldsymbol{\theta}_d) \\
\,\,\,\, = \sum_{d=1}^{D} \sum_{i} x_{di} \Big[(b_i + \mathbf{e}_i^{T} \mathbf{a}_d) - \log \big(\sum_{j}\exp\{b_j + \mathbf{e}_j^{T} \mathbf{a}_d\} \big) \Big]
$


In [ ]:
from scipy.special import log_softmax, logsumexp

class Model:
    """Model defintion, parameters and helper fucntions to compute log-likelihood"""
    
    def __init__(self, D, W, K, doc_embs):
        """Initialize our model
        
        Args:
            D: number of documents
            W: vocab size / number of words
            K: embedding dimension / subspace
        """
        
        self.D = D
        self.W = W
        self.K = K
        
        # word embeddings matrix / subspace 
        n1 = 1. / np.sqrt(K)
        n2 = 1. / np.sqrt(W)
        self.E = np.random.uniform(-n2, n1, size=(W, K))
        
        # bias vector
        self.b = np.random.randn(W, 1)  * 0.001
        
        # document embeddings
        # n3 = 1. / np.sqrt(D)
        # self.A = np.random.uniform(-n1, n3, size=(K, D))
        if doc_embs.shape[1] == K:
            doc_embs = doc_embs.T
        self.A = doc_embs
        
        print('  Model params')
        print('    Word embs:', self.E.shape)
        print('    Doc  embs:', self.A.shape)
        
    def init_bias_with_log_unigram_dist(self, X):
        """We will initialize the bias vector with log of unigram distribution over vocabulary.
        This should help us with better initialization.
        
        b = \log (\sum_d x_d) / (\sum_d \sum_i x_{di})
        """
        
        # if X is sparse matrix, X.A gives the dense version of it in numpy array format
        if isinstance(X, np.ndarray):
            X = X + 1e-08  # to avoid zeros
        else:
            X = X.A + 1e-08  # to avoid any zeros
        
        self.b[:, 0] = np.log(X.sum(axis=0) / X.sum())  # we would like b to of size (W, 1)
        
    def compute_log_thetas(self, sanity_check=False):
        """Compute log of thetas, where theta_d is the unigram distribution over document `d` estiamted from
        the current params (word-embedding matrix, bias vector) and document embedding a_d. """
        
        mat = self.b + (self.E @ self.A)  # shape is W x D
        mat = mat.T  # shape is D x W
        
        # log_norm = logsumexp(mat, axis=1)
        # log_thetas = mat - log_norm
        
        # the following single step is same the two above steps combined
        log_thetas = log_softmax(mat, axis=1)  # shape is D x W
        
        if sanity_check:
            # sanity-check
            # since each document is a proper distribution, it should sum upto 1
            # sum of the matrix should be equal to number of documents
            print("Sanity check for log-thetas:",
                np.allclose(np.exp(log_thetas).sum(), self.D)
            )

        return log_thetas
        
    def compute_log_likelihood(self, X):
        """Compute log-likelihood of the data, given the current parameters / embeddings
        
        Each summation could be implemented using a for-loop but that would very slow, 
        since we have every thing stored in matrices and a sparse matrix, we will do it via
        matrix muliplications and additions.
        
        Args:
            X: doc-by-word counts in scipy.sparse format
            
        Returns:
            float: log-likelihood of the data
        """
        
        log_thetas = self.compute_log_thetas()
        
        # log-likelihood is product of counts to the respective log-probability values.
        if isinstance(X, np.ndarray):
            llh = (X * log_thetas).sum()            
        else:
            # X is a scipy sparse matrix
            llh = (X.multiply(log_thetas)).sum()
        
        return llh
        

### Training the model / updating the parameters

We will keep the bias vector fixed to initial value, which is log of unigram distribution over the vocabulary.

We would like to obtain the parameters that maximize the log-likelihood. We follow gradient-ascent to achieve this target.

We will make these updates alternatively, i.e.,

1. In one-step, we will fix all the document-embeddings $\mathbf{a}_d \, \forall \, d$, and update the word-embedding matrix $\mathbf{E}$.

2. In the next-step, we will update document-embeddings $\mathbf{a}_d \, \forall \, d$, by keeping word-embeddings fixed.


### Obtaining gradients of the objective (log-likelihood) with respect to word embeddings

In the following, we derive the gradient of the log-likelihood $\mathcal{L}$ with respect to the word embeddings, i.e., **each row** $\mathbf{e}_k$ in $\mathbf{E}$.

**NOTE:** $\mathbf{e}_k$ is a row-vector, hence the final gradient will also be a row-vector.

Let us re-write the log-likelihood once again

$\mathcal{L} = \sum_{d=1}^{D} \sum_{i} x_{di} \Big[(b_i + \mathbf{e}_i^{T} \mathbf{a}_d) - \log \big(\sum_{j}\exp\{b_j + \mathbf{e}_j^{T} \mathbf{a}_d\} \big) \Big]
$

$
\nabla_{e_k} \mathcal{L} = \frac{\partial \mathcal{L}}{\partial \mathbf{e}_k} =\frac{\partial  \sum_{d=1}^{D} \sum_{i} x_{di} \Big[(b_i + \mathbf{e}_i^{T} \mathbf{a}_d) - \log \big(\sum_{j}\exp\{b_j + \mathbf{e}_j^{T} \mathbf{a}_d\} \big) \Big]}{\partial \mathbf{e}_k} \\
\qquad \qquad = \sum_d \sum_i x_{di} \Big[ \frac{\partial (b_i + \mathbf{e}_i^{T} \mathbf{a}_d)}{\partial \mathbf{e}_k}   \Big] - \sum_i x_{di} \Big[ \frac{\partial \log \big(\sum_{j}\exp\{b_j + \mathbf{e}_j^{T} \mathbf{a}_d\} \big)}{\partial \mathbf{e}_k} \Big] \\
\qquad \qquad = \sum_d \Big[ x_{dk}(\mathbf{a}_d^T) \Big] - \sum_i x_{di} \Big[ \frac{1}{\sum_j \exp \{b_j + \mathbf{e}_j^T \mathbf{a}_d \}} \big( \frac{\partial \sum_j \exp\{b_j + \mathbf{e}_j^T \mathbf{a}_d\} }{\partial \mathbf{e}_k} \big) \Big] \\
\qquad \qquad = \sum_d \Big[ x_{dk}\mathbf{a}_d^T \Big] - \sum_i x_{di} \Big[ \frac{1}{\sum_j \exp \{b_j + \mathbf{e}_j^T \mathbf{a}_d \}} \big( 0 + \ldots + \exp \{b_k + \mathbf{e}_k^T \mathbf{a}_d \} \mathbf{a}_d^T + 0 + \ldots \big) \Big] \\
\qquad \qquad = \sum_d \Big[ x_{dk}\mathbf{a}_d^T \Big] - \sum_i x_{di} \Big[\frac{ \exp \{b_i + \mathbf{e}_k^T  \mathbf{a}_d^T\}}{\sum_j \exp \{b_j + \mathbf{e}_j^T \mathbf{a}_d \}}\mathbf{a}_d^T  \Big] \\
\qquad \qquad = \sum_d \Big[ x_{dk}\mathbf{a}_d^T \Big] - \sum_i x_{di} \Big[ \theta_{dk} \mathbf{a}_d^T \Big] \\
\qquad \qquad = \sum_d \Big[x_{dk} - (\sum_{i} x_{di}) \theta_{dk} \Big] \mathbf{a}_d^T
$

Interpretation of the gradient (i.e., derivate of log-likelihood $\mathcal{L}$ w.r.t a word embedding for word $k$)

$x_{dk}$: number of times word $k$ appeared in document $d$ 

$\theta_{dk}$: the estimated probability of word $k$ in document $d$

$\sum_i x_{di}$: sum of all the word counts in document $d$


$(\sum_i x_{di}) \theta_{dk}$: weigh the `sum of all word counts` in document $d$ by the probability of word $k$ in $d$, which kind of gives the relative word count of word $k$ in document $d$

$\Big[x_{dk} - (\sum_{i} x_{di}) \theta_{dk} \Big] \mathbf{a}_d^T$: the difference of absoulte word count to the relative word count, and weight this along the direction of document embedding. In the beginning of training this difference is not zero as $\theta_{dk}$ is obatained from random initilization. Towards the end of training, the estimated $\theta_{dk}$ should be close the maximum likelihood $\widehat{\theta}_{dk} = \frac{x_{dk}}{\sum_{i} x_{di}}$

The final gradient is the sum of all weighted document embeddings

In [ ]:
def gradients_E(model, X):
    """Gradient of the log-likelihood with-respect-to word embedding matrix `E`
    
    Args:
        model (Model): The object of the model 
        X (scipy.sparse_matrix): The doc-by-word counts
        
    Returns:
        np.ndarray: Gradient of log-likelihood w.r.t word embeddings, i.e, grad of llh w.r.t to model.E
    """
    
    # grads = np.zeros_like(model.E)  # initialize empty gradients to be the same shape as word embeddings (W, K)
    
    # compute log_thetas as they are needed in gradient
    log_thetas = model.compute_log_thetas()
    
    # the gradient computation can be done using for-loops to reflect the equation
    # or it can be done efficiently using matrix multiplications
    
    # 1. simple way using for-loop    
    # iterate over all documents
    # for d in range(model.D):
        
        # iterate over every word, 
    #     for k in range(model.W):
    #         x_dk = X[d, k]  # count of word k in doc d
    #         rel_x_dk = X[d, :].sum() * np.exp(log_thetas)[d, k]  # relative /estimated count of word k in doc d                  
    #         grads[k, :] += ((x_dk - rel_x_dk) * model.A[:, d])  # doc embeddings are column wise in model.A
            
    # 2. Efficient way of obtaining gradients using matrix operations
    
    ef_grads = np.zeros_like(model.E)
    
    tmp = (X - np.multiply(X.sum(axis=1).reshape(-1, 1), np.exp(log_thetas))).A  # .A will convert matrix to np ndarray
    ef_grads = (model.A @ tmp).T
    
    # Sanity check to see if gradients computed in both ways are numerically identical
    # print('- All close grad_E:', np.allclose(ef_grads, grads))
    
    return ef_grads


In [ ]:
def gradients_A(model, X):
    """Gradient of the log-likelihood with-respect-to document embeddings.
    WE WILL NOT USE IT BECAUSE, WE ALREADY HAVE DOCUMENT/SENTENCE EMBEDDINGS
    FROM PRETRAINED MODEL
    
     Args:
        model (Model): The object of the model 
        X (scipy.sparse_matrix): The doc-by-word counts
        
    Returns:
        np.ndarray: Gradient of log-likelihood w.r.t document embeddings, i.e, grad of llh w.r.t to model.A
    """
    
    grads = np.zeros_like(model.A)  # initialize empty gradients to be the same shape as doc embeddings (K, D)
    
    # compute log_thetas as they are needed in gradient
    log_thetas = model.compute_log_thetas()
    
    
    ##
    ## -- IMPLEMENT THE GRADIENT COMPUTATION HERE (TASK 2 of 4)
    ## The final gradients should be obtained using only matrix multiplications - no for loops
    
    tmp = (X - np.multiply(X.sum(axis=1).reshape(-1, 1), np.exp(log_thetas))).A  # .A will convert matrix to np ndarray
    grads = (tmp @ model.E).T
    
    return grads

In [ ]:
def update_parameters(params, gradient, learning_rate):
    """Update the parameters
    
    Args:
        params (np.ndarray): Word embedding matrix of the document embedding matrix
        gradient (np.ndarray): Gradients of all word embeddings or document embeddings. Should be same as size as params
        learning_rate (float): The learning_rate can also be seen as step size, i.e, the size of the step to be taken
               along the direction of gradient. Too big steps can overshoot our estimate, whereas too small steps
               can take longer for the model to reach optimum.
               
    Returns:
        np.ndarray: the updated params
    """
    
    assert params.shape == gradient.shape, "The params and gradient must have same shape, \
    ({:d}, {:d}) != ({:d} {:d})".format(*params.shape, *gradient.shape)
    
    new_params = params + (learning_rate * gradient)  # since we are doing gradient ascent
    return new_params

In [ ]:
def train(model, X, train_iters, learning_rate):
    """Training scheme for the model"""
    
    llh_0 = model.compute_log_likelihood(X)
    print("  Initial log-likelihood: {:16.2f}".format(llh_0))
    
    llhs = [llh_0]
    
    # pbar = tqdm(np.arange(1, train_iters+1).tolist())
    
    tran = tqdm.trange(train_iters, desc='Log-likelihood {:10.1f} | Training progress'.format(llh_0), leave=True)
    
    for i in tran:       
        
        # First-step: update word embeddings E, by keeping doc-embeddings A fixed
        
        grad_E = gradients_E(model, X)

        model.E = update_parameters(model.E, grad_E, learning_rate['E'])
        
        llh_ei = model.compute_log_likelihood(X)
        # print(" Update E log-likelihood: {:16.2f}".format(llh_ei))
        # pbar.set_description("Log-likelihood: {:16.2f}".format(llh_ei))
        tran.set_description("Log-likelihood {:10.1f} | Training progress".format(llh_ei))
        tran.refresh() # to show immediately the update
        
        if llh_ei < llhs[-1]:
            print("The log-likelihood should improve. Instead it decreased, which means the updates have overshooted, \
decrease the learning_rate (step_size). Additionally, you could make learning_rate as a function of \
iteration so that it decays slowly. Or you could implement back-tracking by halving the learning rate. But this is not \
required for your home-work.")
            break            
        
        llhs.append(llh_ei)
        
        # Next-step (alternating): # First-step: update doc embeddings A, by keeping word-embeddings E fixed
        
        # grad_A = gradients_A(model, X)     
        
        # model.A = update_parameters(model.A, grad_A, learning_rate["A"])
        
        # llh_ai = model.compute_log_likelihood(X) 
        # print(" Update A log-likelihood: {:16.2f}".format(llh_ai))
        
        # if llh_ai < llhs[-1]:
        #    print("The log-likelihood should improve. Instead it decreased, which means the updates have overshooted, \
#decrease the learning_rate (step_size). Additionally, you could make learning_rate as a function of \
#iteration so that it decays slowly. Or you could implement back-tracking by halving the learning rate. But this is not \
#required for your home-work.")
#            break            
        
        # llhs.append(llh_ai)
        
        # learning_rate scheduler
        # we reduce the learning_rate by 10% after every iteration
        learning_rate['E'] -= (learning_rate['E'] * 0.1)
        # learning_rate['A'] -= (learning_rate['A'] * 0.1)
        
    return model, llhs
        

In [ ]:
# We will used only a subset of sentences from strategy 2

count_vect = CountVectorizer(stop_words=None, lowercase=False, strip_accents=None, min_df=1)
DbyW = count_vect.fit_transform(subset_sents)
print('Counts (DbyW)', DbyW.shape)

int2vocab = {}
for word, idx in count_vect.vocabulary_.items():
    int2vocab[idx] = word

np.random.seed(99)

# higher the embedding-dim, more semantic information could be captured, but at the same time
# increases the number of parameters and also training time and memory requriements.

# you could play with emb_dim = {20, 50, 64, 80, 100, 128}

emb_dim = subset_embs.shape[1]  # embedding dim
print('Embedding dim of sentences:', emb_dim)
model = Model(DbyW.shape[0], DbyW.shape[1], emb_dim, subset_embs)

initial_llh = model.compute_log_likelihood(DbyW)
print("Initial  log-likelihood:", initial_llh)

print("Initializing bias vector to log-unigram distribution.")
model.init_bias_with_log_unigram_dist(DbyW)

new_init_llh = model.compute_log_likelihood(DbyW)
print("New init log-likelihood:", new_init_llh)

In [ ]:
train_iters = 50

# we could use different learning_rate for word and document embeddings
# smaller learning rates can ensure that updates do not overshoot, but at the same time, it can many iterations
# for the model to converge

# higher learning rate can make big steps, but can overshoot, one can use backtracking approach to make smaller steps.

# You can play with various learning rate and see how quickly the model converges
learning_rate = {'E': 0.0005, 'A': 0.001}

In [ ]:
# Let's train the model
# Run this cell again to continue traning for more iterations
os.environ['OMP_NUM_THREADS'] = '4'
os.environ['MKL_NUM_THREADS'] = '4'
model, llhs = train(model, DbyW, train_iters, learning_rate)

In [ ]:
def plot_log_likelihood(llhs, fig_num):
        
    plt.figure(fig_num)
    plt.plot(np.arange(0, len(llhs)/2., 0.5), llhs, '.-')
    
    plt.xlabel('Training iterations')
    plt.ylabel('Log-likelihood')
    
    plt.grid(alpha=0.5, linestyle='--')
    plt.show()
    
plot_log_likelihood(llhs, 10)

In [ ]:
# Cluster and Get more representative words using the trained word embeddings

def cluster_and_get_representative_words(int2vocab, model, n_clusters):

    embs = model.A.T
    print('Doc embeddings (model.A.T):', model.A.T.shape)

    print("Clustering", embs.shape[0], "embeddings into", n_clusters, "clusters with k-means..")

    norm_embs = preprocessing.normalize(embs)
    print("  Normalizing embeddings to have unit length..")

    kmeans2 = KMeans(n_clusters=n_clusters, random_state=12345, max_iter=1000)
    pred_ixs2 = kmeans2.fit_predict(norm_embs)
    print('.. done')
    
    cluster_ixs, cluster_strength = np.unique(pred_ixs2, return_counts=True)

    cluster_strength = cluster_strength / pred_ixs2.shape[0]

    centroids = kmeans2.cluster_centers_

    print("Cluster centers:", centroids.shape)

    global_mean = np.mean(embs, axis=0)

    scores = (model.E @ (centroids - global_mean).T) 
    # scores = (model.E @ centroids.T) 

    topn = 15  # top N words per cluster

    print("Cluster_index (strength in %) Representative tokens\n")
    for k in range(n_clusters):
        k_ixs = np.argsort(scores[:, k])[::-1] # sort in descending order
        print("{:3d} ({:3.1f})".format(k, cluster_strength[k]*100.), end="   ")
        for i in range(topn):
            print(int2vocab[k_ixs[i]], end=", ")
        print("\n")
        
    print()
    
#     top_k = 15  # print TOP K words per cluster    
#     for i in range(n_clusters):
#         sort_ixs = np.argsort(prob_per_cluster[i, :].A.squeeze())[::-1]
#         print("= Cluster", i, end="=\n ")
#         for j in range(top_k):
#             print(int2vocab[sort_ixs[j]], end=", ")
#         print("\n")
        
    return pred_ixs2

In [ ]:
pred_ixs2 = cluster_and_get_representative_words(int2vocab, model, 12)

In [ ]:
trans2, soc2, eoc2 = compute_transitions(subset_domain_convs, pred_ixs2)  # strategy 2 with subset
print("Transition matrix:", trans2.shape)
for i in range(trans2.shape[0]):
    trans2[i, :] /= trans2[i, :].sum()
    
print("\nStart of conversation..")
# get the clusters where the conversation usually BEGIN
soc_prob2, soc_ixs2 = get_cluster_assignments(soc2)


print("\nEnd of conversation..")
# get the clusters where the conversation usually END
eoc_prob2, eoc_ixs2 = get_cluster_assignments(eoc2)

In [ ]:
dot2 = visualize_graph(trans2, soc_prob2, eoc_prob2)
dot2

In [ ]:
get_most_representative_words(subset_sents, pred_ixs2)